## Slurm Experiment Running

I wrote most of the key experimental organization tools to the ```sd_bandits.utils```directory. Note that I'm using the ```sd_bandits_env``` defined in ```conda_envs/sd_bandits.yml```. Let's start by importing this:

In [2]:
import sys
import os
root_dir = os.path.expanduser('~/sd_bandits')
sys.path.append(root_dir)

import sd_bandits.utils as utils

The key functions are:
* ```build_obj_spec()```, which builds a .yml file for a specific policy, estimator, or dataset
* ```load_obj_from_spec()```, which loads a dictionary from the .yml file created with the above function
* ```build_experiment()```, which builds a policy, estimator, and dataset, and constructs a slurm batch submission script

Let's say I wanted to test this out on the full ```obp``` dataset w/ the bts policy data:

In [18]:
dataset_name = 'obp'
#this is all the arguments I'd pass to the OBP dataset constructor
dataset_params = {'data_path': '~/sd_bandits/data/obd',
                       'campaign': 'all',
                       'behavior_policy': 'bts'}

dataset_dict = utils.build_obj_spec(dataset_name, dataset_params, experiment_name='notebook_demo_1',
                                    obj_type='dataset', output='./policy_yamls/')
print(dataset_dict)

{'name': 'notebook_demo_1', 'type': 'dataset', 'key': 'obp', 'parameters': {'data_path': '~/sd_bandits/data/obd', 'campaign': 'all', 'behavior_policy': 'bts'}}


I've also saved this tp ```./policy_yamls/notebook_demo_1/dataset_sepc.yml```. I can load the dataset by:

In [10]:
dataset = utils.load_obj_from_spec('./policy_yamls/notebook_demo_1/dataset_spec.yaml')
print(dataset)

OpenBanditDataset(behavior_policy='bts', campaign='all', data_path=PosixPath('~/sd_bandits/data/obd/bts/all'), dataset_name='obd')


I can repeat this for a policy and estimator. For sake of example, I'll do a ```LogisticTS``` policy and a ```DoublyRobustWithShrinkage``` estimator

In [21]:
policy_name = 'LogisticTS'
policy_params = {'n_actions':dataset.n_actions, 'len_list':dataset.len_list, 'dim':dataset.context.shape[1]}
policy_dict = utils.build_obj_spec(policy_name, policy_params, experiment_name='notebook_demo_1',
                                    obj_type='policy' , output='./policy_yamls/')

estimator_name = 'DoublyRobustWithShrinkage'
estimator_params = {'lambda_':1}
estimator_dict = utils.build_obj_spec(estimator_name, estimator_params, experiment_name='notebook_demo_1',
                                      obj_type='estimator' , output='./policy_yamls/')

Now, I can run this experiment using the script ```opb_run.py --experiment-dir 'policy_yamls/notebook_demo_1/```, and the resulting interval estimates would be saved in the same directory. For convenience, theres a single function that can generate all three objects, and a script for submitting a job to slurm:

In [24]:
experiment_test_name = 'notebook_demo_2'

utils.build_experiment(experiment_test_name, policy_name, estimator_name, dataset_name,\
                       policy_params, estimator_params, dataset_params)

From here, ```sbatch ./policy_yamls/notebook_demo_2/script.sbatch``` will submit the job to a slurm node.